In [1]:
import torch
import numpy as np
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW 
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertModel
from transformers import BertConfig

##回傳類別
def get_tag(str_in):
    pretrain_model_path = '/home/nlp/NewsClassify/bert_pretrain_news/'
    model_path = '/home/nlp/NewsClassify/'
    ##新聞分類
    NUM_LABELS = 7
    tokenizer = BertTokenizer.from_pretrained(pretrain_model_path)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = BertForSequenceClassification.from_pretrained(pretrain_model_path,num_labels = NUM_LABELS)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path+'BERT_for_xgboost_0.pkl'), strict=False)

    model.eval()
    model1 = BertForSequenceClassification.from_pretrained(pretrain_model_path,num_labels = NUM_LABELS)
    model1 = model1.to(device)
    model1.load_state_dict(torch.load(model_path+'BERT_for_xgboost_1.pkl'), strict=False)
    model1.eval()

    model2 = BertForSequenceClassification.from_pretrained(pretrain_model_path,num_labels = NUM_LABELS)
    model2 = model2.to(device)
    model2.load_state_dict(torch.load(model_path+'BERT_for_xgboost_2.pkl'), strict=False)
    model2.eval()
    model_list = [model,model1,model2] ##ensemble用
    class TestDataset(Dataset):
        def __init__(self, input_dict):
            self.input_ids = input_dict['input_ids']
            self.token_type_ids = input_dict['token_type_ids']
            self.attention_mask = input_dict['attention_mask']
        def __getitem__(self, idx):
            input_id = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            return input_id, tokentype, attentionmask
        def __len__(self):
            return len(self.input_ids)
    tag_map = ['政治','生活','國際','體育','娛樂','社會','財經']
    score_list = [0,0,0,0,0,0,0]
    tag_list = []
    threshold = 3
    str_in = [str_in] #list
    test_input_dict = tokenizer.batch_encode_plus(str_in,
                                             add_special_tokens = True,
                                             max_length = 512,
                                             truncation = True,
                                             return_special_tokens_mask = True,
                                             pad_to_max_length = True,
                                             return_tensors = 'pt')
    testset = TestDataset(test_input_dict)
    testloader = DataLoader(testset, batch_size = 1)
    for data in testloader:
        tokens_tensors , segment_tensors,masks_tensors = [t.to(device) for t in data]
        for model in model_list:
            outputs = model(input_ids = tokens_tensors,
                            token_type_ids = segment_tensors,
                            attention_mask = masks_tensors)
            print(outputs[0][0])
            soft = F.Softmax(dim=-1)
            test = soft(outputs[0][0])
            print(test)
            for idx in range(NUM_LABELS):
                score_list[idx] += outputs[0][0][idx]
        for idx in range(len(score_list)):
            if score_list[idx] >= threshold:
                tag_list.append(tag_map[idx])
    return tag_list  

## 回傳報社
def get_news(str_in):
    pretrain_model_path = '/home/nlp/NewsClassify/bert_pretrain_news/'
    model_path = '/home/nlp/NewsClassify/'
    ##報社分類
    LABELS = 3
    tokenizer = BertTokenizer.from_pretrained(pretrain_model_path)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    news_model = BertForSequenceClassification.from_pretrained(pretrain_model_path,num_labels = LABELS)
    news_model = news_model.to(device)
    news_model.load_state_dict(torch.load(model_path+'BERT_news_2e5_0.pkl'), strict=False)
    news_model.eval()
    class TestDataset(Dataset):
        def __init__(self, input_dict):
            self.input_ids = input_dict['input_ids']
            self.token_type_ids = input_dict['token_type_ids']
            self.attention_mask = input_dict['attention_mask']
        def __getitem__(self, idx):
            input_id = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            return input_id, tokentype, attentionmask
        def __len__(self):
            return len(self.input_ids)
    tag_map = ['中國時報','自由時報','聯合報']
    tag_list = []
    str_in = [str_in] #list
    test_input_dict = tokenizer.batch_encode_plus(str_in,
                                             add_special_tokens = True,
                                             max_length = 512,
                                             truncation = True,
                                             return_special_tokens_mask = True,
                                             pad_to_max_length = True,
                                             return_tensors = 'pt')
    testset = TestDataset(test_input_dict)
    testloader = DataLoader(testset, batch_size = 1)
    for data in testloader:
        tokens_tensors , segment_tensors,masks_tensors = [t.to(device) for t in data]
        outputs = news_model(input_ids = tokens_tensors,
                        token_type_ids = segment_tensors,
                        attention_mask = masks_tensors)
        pred_idx = torch.argmax(outputs[0],dim = -1)
        tag_list.append(tag_map[pred_idx])
    return tag_list  


In [2]:
test = ''
ans = get_tag(test)
news_ans = get_news(test)
print(ans)
print(news_ans)

Some weights of the model checkpoint at /home/nlp/NewsClassify/bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassif

tensor([ 0.3495,  6.2224, -2.0108, -1.8810, -1.1850, -0.9934, -0.3400],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([2.7977e-03, 9.9390e-01, 2.6408e-04, 3.0067e-04, 6.0309e-04, 7.3043e-04,
        1.4040e-03], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([-0.1562,  5.6079, -2.4652, -0.6493, -2.2486,  0.8467, -1.2622],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([3.0906e-03, 9.8488e-01, 3.0710e-04, 1.8875e-03, 3.8135e-04, 8.4260e-03,
        1.0227e-03], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([ 2.1841,  4.3486, -2.4497, -2.2741, -0.4747, -0.5789,  0.6374],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.0992, 0.8643, 0.0010, 0.0011, 0.0069, 0.0063, 0.0211],
       device='cuda:0', grad_fn=<SoftmaxBackward>)


Some weights of the model checkpoint at /home/nlp/NewsClassify/bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassif

['生活']
['自由時報']
